In [1]:
import os
os.chdir("..")

In [2]:
import pickle
import pandas as pd
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from typing import Callable
from src.config import Config
from src.dataset import Interactions
from src.metrics import PrecisionAtK, RecallAtK
from src.optimizer import OptunaMaximizer

In [3]:
with Config.PREPROCESSED_INTERACTIONS_PATH.open("rb") as file_object:
    interactions = pickle.load(file_object)

In [4]:
interactions["interest_score_binary"] = 0
interactions.loc[interactions["interest_score" ]>= .7, ["interest_score_binary"]] = 1

In [5]:
interactions

,user_id,item_id,progress,rating,start_date,interest_score,interest_score_binary
0,126706,14433,80,NaN,2018-01-01,0.400,0
1,127290,140952,58,NaN,2018-01-01,0.290,0
2,66991,198453,89,NaN,2018-01-01,0.445,0
3,46791,83486,23,5.0,2018-01-01,0.615,0
4,79313,188770,88,5.0,2018-01-01,0.940,1
...,...,...,...,...,...,...,...
1532993,153908,98585,44,NaN,2019-02-11,0.220,0
1532994,154008,251969,4,NaN,2018-04-08,0.020,0
1532995,154892,298192,68,5.0,2019-02-16,0.840,1
1532996,156948,38118,78,5.0,2018-08-19,0.890,1


In [6]:
interactions = Interactions(interactions, test_days=30, valid_days=60)

In [7]:
interactions.train

,user_id,item_id,progress,rating,start_date,interest_score,interest_score_binary
0,126706,14433,80,NaN,2018-01-01,0.400,0
1,127290,140952,58,NaN,2018-01-01,0.290,0
2,66991,198453,89,NaN,2018-01-01,0.445,0
3,46791,83486,23,5.0,2018-01-01,0.615,0
4,79313,188770,88,5.0,2018-01-01,0.940,1
...,...,...,...,...,...,...,...
1532993,153908,98585,44,NaN,2019-02-11,0.220,0
1532994,154008,251969,4,NaN,2018-04-08,0.020,0
1532995,154892,298192,68,5.0,2019-02-16,0.840,1
1532996,156948,38118,78,5.0,2018-08-19,0.890,1


In [8]:
interactions.train = pd.concat(
    [
        interactions.train,
        interactions.test[interactions.test["interest_score_binary"] == 0].copy(),
        interactions.valid[interactions.valid["interest_score_binary"] == 0].copy()
    ]
)
interactions.train

,user_id,item_id,progress,rating,start_date,interest_score,interest_score_binary
0,126706,14433,80,NaN,2018-01-01,0.400,0
1,127290,140952,58,NaN,2018-01-01,0.290,0
2,66991,198453,89,NaN,2018-01-01,0.445,0
3,46791,83486,23,5.0,2018-01-01,0.615,0
4,79313,188770,88,5.0,2018-01-01,0.940,1
...,...,...,...,...,...,...,...
1469712,142744,230067,3,NaN,2019-12-01,0.015,0
1469713,73145,223263,6,NaN,2019-12-01,0.030,0
1469715,100060,313623,11,5.0,2019-12-01,0.555,0
1469717,82749,165018,2,NaN,2019-12-01,0.010,0


In [9]:
interactions.test = interactions.test[interactions.test["interest_score_binary"] == 1]
interactions.test

,user_id,item_id,progress,rating,start_date,interest_score,interest_score_binary
1469727,129896,168726,100,5.0,2019-12-02,1.000,1
1469756,89727,75814,75,5.0,2019-12-02,0.875,1
1469772,49785,252986,99,5.0,2019-12-02,0.995,1
1469775,135061,141047,49,5.0,2019-12-02,0.745,1
1469777,47747,55458,99,3.0,2019-12-02,0.795,1
...,...,...,...,...,...,...,...
1530792,113785,230592,100,5.0,2019-12-30,1.000,1
1530793,49989,234938,99,5.0,2019-12-30,0.995,1
1530794,96202,230592,100,5.0,2019-12-30,1.000,1
1530799,29877,162000,79,5.0,2019-12-30,0.895,1


In [10]:
interactions.valid = interactions.valid[interactions.valid["interest_score_binary"] == 1]
interactions.valid

,user_id,item_id,progress,rating,start_date,interest_score,interest_score_binary
1406113,135781,266343,85,5.0,2019-11-02,0.925,1
1406118,140910,136327,100,5.0,2019-11-02,1.000,1
1406122,33450,109201,98,5.0,2019-11-02,0.990,1
1406130,159354,78547,91,5.0,2019-11-02,0.955,1
1406148,83685,168854,99,3.0,2019-11-02,0.795,1
...,...,...,...,...,...,...,...
1469657,2681,70828,86,5.0,2019-12-01,0.930,1
1469667,3951,239790,99,5.0,2019-12-01,0.995,1
1469689,10808,72830,88,5.0,2019-12-01,0.940,1
1469701,45181,310618,73,5.0,2019-12-01,0.865,1


In [11]:
interactions.train = interactions.train[["user_id", "item_id", "interest_score_binary"]]
interactions.test = interactions.test[["user_id", "item_id", "interest_score_binary"]]
interactions.valid = interactions.valid[["user_id", "item_id", "interest_score_binary"]]

In [12]:
csr_train = csr_matrix(
    (interactions.train["interest_score_binary"], 
    (interactions.train["user_id"], interactions.train["item_id"]))
)
csr_train

<159613x321752 sparse matrix of type '<class 'numpy.int64'>'
	with 1316432 stored elements in Compressed Sparse Row format>

In [13]:
class Objective:
    def __init__(self, train: csr_matrix, valid: pd.DataFrame, metric: Callable):
        self.train = train
        self.valid = valid
        self.metric = metric

    def __call__(self, trial) -> float:
        search_space = {
            "factors": trial.suggest_int("factors", 4, 256),
            "regularization": trial.suggest_float("regularization", 1e-8, 0.1),
            "iterations": trial.suggest_int("iterations", 8, 64),
            "num_threads": Config.NUM_THREADS
        }
        model = AlternatingLeastSquares(**search_space)
        model.fit(self.train)
        predictions = pd.DataFrame(self.valid["user_id"].drop_duplicates())
        predictions["item_id"] = predictions["user_id"].apply(
            lambda user_id: 
                model.recommend(
                    user_id, 
                    self.train[user_id],
                    N=Config.K,
                    filter_already_liked_items=True
                )[0]
        )
        predictions = predictions.explode("item_id")
        return self.valid["user_id"].drop_duplicates().apply(
            lambda user_id: 
                self.metric.calculate(
                    predictions[predictions["user_id"] == user_id]["item_id"],
                    self.valid[self.valid["user_id"] == user_id]["item_id"]
                )
        ).mean()

In [14]:
recall = RecallAtK(Config.K)
precision = PrecisionAtK(Config.K)

In [15]:
precision_objective = Objective(csr_train, interactions.valid, precision)
precision_optimizer = OptunaMaximizer(precision_objective, "ALS precision maximize")
precision_best_trial = precision_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-03-11 12:58:50,108] A new study created in memory with name: ALS precision maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-03-11 12:58:55,849] Trial 0 finished with value: 0.0037897586284683037 and parameters: {'factors': 126, 'regularization': 0.022618699036281446, 'iterations': 10}. Best is trial 0 with value: 0.0037897586284683037.


  0%|          | 0/34 [00:00<?, ?it/s]

...

[I 2023-03-11 13:02:42,430] Trial 20 finished with value: 0.0022106925332731767 and parameters: {'factors': 9, 'regularization': 0.052451428594913314, 'iterations': 24}. Best is trial 2 with value: 0.004331152718249495.


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2023-03-11 13:02:57,070] Trial 21 finished with value: 0.00399278141213625 and parameters: {'factors': 253, 'regularization': 0.005360198495633323, 'iterations': 40}. Best is trial 2 with value: 0.004331152718249495.


  0%|          | 0/46 [00:00<?, ?it/s]

[I 2023-03-11 13:03:11,908] Trial 22 finished with value: 0.0041958041958041975 and parameters: {'factors': 250, 'regularization': 0.001987748484152738, 'iterations': 46}. Best is trial 2 with value: 0.004331152718249495.
EarlyStopping Exceeded: No new best scores on iters 20


In [16]:
recall_objective = Objective(csr_train, interactions.valid, recall)
recall_optimizer = OptunaMaximizer(recall_objective, "ALS recall maximize")
recall_best_trial = recall_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-03-11 13:03:11,995] A new study created in memory with name: ALS recall maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

[I 2023-03-11 13:03:22,314] Trial 0 finished with value: 0.010714116914709991 and parameters: {'factors': 159, 'regularization': 0.08825201092591779, 'iterations': 53}. Best is trial 0 with value: 0.010714116914709991.


  0%|          | 0/25 [00:00<?, ?it/s]

...

[I 2023-03-11 13:10:49,245] Trial 53 finished with value: 0.014454096400862737 and parameters: {'factors': 198, 'regularization': 0.06683824093610942, 'iterations': 15}. Best is trial 35 with value: 0.017430806262161168.


  0%|          | 0/12 [00:00<?, ?it/s]

[I 2023-03-11 13:10:57,925] Trial 54 finished with value: 0.014726191717370668 and parameters: {'factors': 237, 'regularization': 0.07069193279712754, 'iterations': 12}. Best is trial 35 with value: 0.017430806262161168.


  0%|          | 0/47 [00:00<?, ?it/s]

[I 2023-03-11 13:11:10,545] Trial 55 finished with value: 0.012952984349757708 and parameters: {'factors': 224, 'regularization': 0.0782656620644333, 'iterations': 47}. Best is trial 35 with value: 0.017430806262161168.
EarlyStopping Exceeded: No new best scores on iters 20


In [17]:
precision_model = AlternatingLeastSquares(**precision_best_trial.params)
precision_model.fit(csr_train)

  0%|          | 0/38 [00:00<?, ?it/s]

In [18]:
recall_model = AlternatingLeastSquares(**recall_best_trial.params)
recall_model.fit(csr_train)

  0%|          | 0/11 [00:00<?, ?it/s]

In [19]:
models = {
    "recall_model": recall_model,
    "precision_model": precision_model
}

In [20]:
metrics = {
    "precision": precision,
    "recall": recall
}

In [21]:
for model_name, model in models.items():
    pred = pd.DataFrame(interactions.test["user_id"].drop_duplicates())
    print(f"Model '{model_name}':")
    pred["item_id"] = pred["user_id"].apply(
        lambda user_id: 
            model.recommend(
                user_id, 
                csr_train[user_id],
                N=Config.K,
                filter_already_liked_items=True
            )[0]
    )
    pred = pred.explode("item_id")
    for metric_name, metric in metrics.items():
        metric_val = interactions.test["user_id"].drop_duplicates().apply(
            lambda user_id: 
                metric.calculate(
                    pred[pred["user_id"] == user_id]["item_id"],
                    interactions.test[interactions.test["user_id"] == user_id]["item_id"]
                )
        ).mean()
        print(f"\tMetric {metric_name}@{Config.K}: {metric_val}")

Model 'recall_model':
	Metric precision@10: 0.0023210831721469983
	Metric recall@10: 0.008298372726564213
Model 'precision_model':
	Metric precision@10: 0.0019825918762088955
	Metric recall@10: 0.006664199736024153


In [22]:
with (Config.MODEL_PATH / "als_binary.pickle").open("wb") as file_object:
    pickle.dump(recall_model, file_object)